<a href="https://colab.research.google.com/github/ZahraShahlaie/Crack-detection/blob/main/IAAA2end_Medical_SAM_LSTM_image_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, we use features extracted from the Medical SAM 2 model, which are stored as embeddings with dimensions `(1, 256, 64, 64)`. After initial processing using Adaptive Average Pooling and Flattening methods, the dimensions of the embeddings are reduced to `(1, 1024)`. These compressed embeddings are now ready to be utilized in machine learning models or for further analysis.

In this notebook, we use the compressed embeddings with dimensions `(1, 1024)`.



# Load dataset and install requirements

In [1]:
!gdown 1R_T8FKXorobu_4aDwrt4qFf3tVVEtaSr
!unzip -q v11_MedSAM_embeddings_V2.zip -d .


Downloading...
From (original): https://drive.google.com/uc?id=1R_T8FKXorobu_4aDwrt4qFf3tVVEtaSr
From (redirected): https://drive.google.com/uc?id=1R_T8FKXorobu_4aDwrt4qFf3tVVEtaSr&confirm=t&uuid=fb86a27f-b8d3-456c-b1f5-396a76c3051f
To: /content/v11_MedSAM_embeddings_V2.zip
100% 278M/278M [00:03<00:00, 82.1MB/s]


In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchsummary import summary
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import auc, roc_curve, precision_recall_curve, roc_auc_score
import pandas as pd
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm

In [3]:
!mkdir epoch_FLAIR
!mkdir epoch_T1
!mkdir epoch_T2

# Model and Dataset

In [4]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)  # Set batch_first=True
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Reshape the input to (batch_size, sequence_length, input_size)
        x = x.view(x.size(0), -1, self.lstm.input_size)  # Flatten channels, height, and width into sequence_length
        h0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to('cuda')  # Initial hidden state
        c0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to('cuda')  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))  # LSTM
        out = self.fc(out[:, -1, :])  # Fully connected layer
        out = self.sigmoid(out)  # Sigmoid activation for binary classification
        return out


class EmbeddingDataset(Dataset):
    def __init__(self, df, protocol, mode, window_size=16):
        self.df = df
        self.protocol = protocol
        self.mode = mode
        self.window_size = window_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get the ID and load the corresponding npz file
        id = self.df["SeriesInstanceUID"][idx]
        feature_map_path = f'/content/embeddings/{self.protocol}/{self.mode}/{id}.npz'
        npz_data = np.load(feature_map_path)
        embeddings = npz_data['embeddings']

        # Get the sequence length
        sequence_length = embeddings.shape[0]

        # If sequence length is greater than window_size, keep the middle window_size sequences
        if sequence_length > self.window_size:
            start_idx = (sequence_length - self.window_size) // 2
            embeddings = embeddings[start_idx:start_idx + self.window_size]
        elif sequence_length < self.window_size:
            # Padding: pad sequence to window_size
            padding_length = self.window_size - sequence_length
            embeddings = np.pad(embeddings, ((0, padding_length), (0, 0)), mode='constant', constant_values=0)

        # Convert to PyTorch tensor
        embeddings = torch.tensor(embeddings, dtype=torch.float32)

        # Get the label and convert to PyTorch tensor
        label = self.df['prediction'][idx]
        label = torch.tensor(label, dtype=torch.float32)

        return embeddings, label


# Train Loop

In [5]:
def train(modality, model, optimizer, criterion, train_loader, val_loader):
    for epoch in range(NUM_EPOCHS):
        model.train()  # Set the model to training mode
        all_train_loss = 0.0
        all_pred_train = []
        all_targets_train = []
        for input, targets in tqdm(train_loader, desc=f'Epoch {epoch+1} train'):
            optimizer.zero_grad()  # Zero the gradients
            input, targets = input.to("cuda"), targets.to("cuda")
            outputs = model(input)  # Forward pass
            # Reshape targets only if necessary
            if outputs.shape != targets.shape:
                targets = targets.unsqueeze(1)
            loss = criterion(outputs, targets)  # Compute the loss

            # Append the flattened arrays to the lists
            all_targets_train.append(targets.cpu().detach().numpy().flatten()) # Flatten the array
            all_pred_train.append(outputs.cpu().detach().numpy().flatten()) # Flatten the array
            loss.backward()  # Backward pass
            optimizer.step()  # Update the weights

            all_train_loss += loss.item() * input.size(0) # Accumulate the loss

        model.eval()
        val_all_labels=[]
        val_all_preds = []
        all_val_loss = 0.0
        with torch.no_grad():
            for input, targets in tqdm(val_loader, desc=f'Epoch {epoch+1} validation'):
                input, targets = input.to("cuda"), targets.to("cuda")
                outputs = model(input)  # Forward pass

                # Reshape targets only if necessary
                if outputs.shape != targets.shape:
                    targets = targets.unsqueeze(1)

                loss = criterion(outputs, targets)  # Compute the loss
                val_all_labels.append(targets.cpu().detach().numpy().flatten())
                val_all_preds.append(outputs.cpu().detach().numpy().flatten())
                all_val_loss += loss.item() * input.size(0) # Accumulate the loss


        # Train Metrics
        all_targets_train = np.concatenate(all_targets_train) # Concatenate into a 1D array
        all_pred_train = np.concatenate(all_pred_train) # Concatenate into a 1D array

        fpr, tpr, thresholds_roc = roc_curve(all_targets_train, all_pred_train)
        precision, recall, thresholds_pr = precision_recall_curve(all_targets_train, all_pred_train)

        train_loss = all_train_loss / len(train_loader.dataset) # loss
        train_auc = auc(fpr, tpr) # AUC
        pr_auc = auc(recall, precision) # AUPRC

        print(f'TRAIN ->\t LOSS: {train_loss:.4f},\t AUC: {train_auc:.4f},\t AUPRC: {pr_auc:.4f}')

        # Validation Metrics
        val_all_labels = np.concatenate(val_all_labels) # Concatenate into a 1D array
        val_all_preds = np.concatenate(val_all_preds) # Concatenate into a 1D array

        fpr, tpr, thresholds_roc = roc_curve(val_all_labels, val_all_preds)
        precision, recall, thresholds_pr = precision_recall_curve(val_all_labels, val_all_preds)

        val_loss = all_val_loss / len(val_loader.dataset) # loss
        val_auc = auc(fpr, tpr) # AUC
        pr_auc = auc(recall, precision) # AUPRC

        print(f"VALIDATION ->\t LOSS: {val_loss:.4f},\t AUC: {val_auc:.4f},\t AUPRC: {pr_auc:.4f}")

        dummy_input = torch.randn(1, 1024).to("cuda")
        traced_model = torch.jit.trace(model, dummy_input)
        traced_model.save(f"/content/epoch_{modality}/epoch-{epoch+1}.pth")

        print()

# Train models

## FLAIR train

In [6]:
!rm -rf /content/epoch_FLAIR/*

In [7]:
BATHC_SIZE = 16
NUM_EPOCHS = 500
protocol = "FLAIR"

df_train_data = pd.read_csv(f"/content/embeddings/{protocol}/training/info.csv")
train_dataset = EmbeddingDataset(df_train_data, protocol, 'training')
train_loader = DataLoader(train_dataset, batch_size=BATHC_SIZE, shuffle=True)

df_val_data = pd.read_csv(f"/content/embeddings/{protocol}/validation/info.csv")
val_dataset = EmbeddingDataset(df_val_data, protocol, 'validation')
val_loader = DataLoader(val_dataset, batch_size=BATHC_SIZE, shuffle=True)

# Hyperparameters
input_size = 256*4
hidden_size = 256*2
output_size = 1
learning_rate = 0.0001

# Model, loss function, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_size, hidden_size, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# sample_input = torch.randn(1, 3, 224, 224).to("cuda")
# summary(obj_model, input_size=(3, 224, 224), device="cuda")

all_params = sum(p.numel() for p in model.parameters())
print('-'*40)
print(f'All parameters: {all_params:,.0f}')
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params:,.0f}')
print('-'*40)
print()

train(protocol, model, optimizer, criterion, train_loader, val_loader)

----------------------------------------
All parameters: 3,150,337
Trainable parameters: 3,150,337
----------------------------------------



Epoch 1 validation: 100%|██████████| 17/17 [00:00<00:00, 53.71it/s]


TRAIN ->	 LOSS: 0.4198,	 AUC: 0.4799,	 AUPRC: 0.1262
VALIDATION ->	 LOSS: 0.3872,	 AUC: 0.4942,	 AUPRC: 0.1353



Epoch 2 validation: 100%|██████████| 17/17 [00:00<00:00, 61.70it/s]


TRAIN ->	 LOSS: 0.3841,	 AUC: 0.4695,	 AUPRC: 0.1327
VALIDATION ->	 LOSS: 0.3820,	 AUC: 0.5589,	 AUPRC: 0.1481



...

TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 2.0772,	 AUC: 0.7224,	 AUPRC: 0.2783



Epoch 499 validation: 100%|██████████| 17/17 [00:00<00:00, 55.91it/s]


TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 2.0994,	 AUC: 0.7217,	 AUPRC: 0.2765



Epoch 500 validation: 100%|██████████| 17/17 [00:00<00:00, 54.87it/s]

TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 2.1029,	 AUC: 0.7225,	 AUPRC: 0.2777



## T1 train

In [8]:
!rm -rf /content/epoch_T1/*

In [9]:
BATHC_SIZE = 16
NUM_EPOCHS = 500
protocol = "T1"

df_train_data = pd.read_csv(f"/content/embeddings/{protocol}/training/info.csv")
train_dataset = EmbeddingDataset(df_train_data, protocol, 'training')
train_loader = DataLoader(train_dataset, batch_size=BATHC_SIZE, shuffle=True)

df_val_data = pd.read_csv(f"/content/embeddings/{protocol}/validation/info.csv")
val_dataset = EmbeddingDataset(df_val_data, protocol, 'validation')
val_loader = DataLoader(val_dataset, batch_size=BATHC_SIZE, shuffle=True)

# Hyperparameters
input_size = 256*4
hidden_size = 256*2
output_size = 1
learning_rate = 0.0001

# Model, loss function, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_size, hidden_size, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# sample_input = torch.randn(1, 3, 224, 224).to("cuda")
# summary(obj_model, input_size=(3, 224, 224), device="cuda")

all_params = sum(p.numel() for p in model.parameters())
print('-'*40)
print(f'All parameters: {all_params:,.0f}')
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params:,.0f}')
print('-'*40)
print()

train(protocol, model, optimizer, criterion, train_loader, val_loader)

----------------------------------------
All parameters: 3,150,337
Trainable parameters: 3,150,337
----------------------------------------



Epoch 1 validation: 100%|██████████| 17/17 [00:00<00:00, 28.71it/s]


TRAIN ->	 LOSS: 0.3867,	 AUC: 0.4919,	 AUPRC: 0.1543
VALIDATION ->	 LOSS: 0.3694,	 AUC: 0.6483,	 AUPRC: 0.2029



Epoch 2 validation: 100%|██████████| 17/17 [00:00<00:00, 63.20it/s]


TRAIN ->	 LOSS: 0.3691,	 AUC: 0.4552,	 AUPRC: 0.1102
VALIDATION ->	 LOSS: 0.3690,	 AUC: 0.6743,	 AUPRC: 0.2160



...

TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 2.4584,	 AUC: 0.6285,	 AUPRC: 0.1696



Epoch 499 validation: 100%|██████████| 17/17 [00:00<00:00, 54.23it/s]


TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 2.6075,	 AUC: 0.6231,	 AUPRC: 0.1653



Epoch 500 validation: 100%|██████████| 17/17 [00:00<00:00, 59.83it/s]


TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 2.4999,	 AUC: 0.6278,	 AUPRC: 0.1684



##T2_train

In [10]:
!rm -rf /content/epoch_T2/*

In [11]:
BATHC_SIZE = 16
NUM_EPOCHS = 500
protocol = "T2"

df_train_data = pd.read_csv(f"/content/embeddings/{protocol}/training/info.csv")
train_dataset = EmbeddingDataset(df_train_data, protocol, 'training')
train_loader = DataLoader(train_dataset, batch_size=BATHC_SIZE, shuffle=True)

df_val_data = pd.read_csv(f"/content/embeddings/{protocol}/validation/info.csv")
val_dataset = EmbeddingDataset(df_val_data, protocol, 'validation')
val_loader = DataLoader(val_dataset, batch_size=BATHC_SIZE, shuffle=True)

# Hyperparameters
input_size = 256*4
hidden_size = 256*2
output_size = 1
learning_rate = 0.0001

# Model, loss function, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_size, hidden_size, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# sample_input = torch.randn(1, 3, 224, 224).to("cuda")
# summary(obj_model, input_size=(3, 224, 224), device="cuda")

all_params = sum(p.numel() for p in model.parameters())
print('-'*40)
print(f'All parameters: {all_params:,.0f}')
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params:,.0f}')
print('-'*40)
print()

train(protocol, model, optimizer, criterion, train_loader, val_loader)

----------------------------------------
All parameters: 3,150,337
Trainable parameters: 3,150,337
----------------------------------------



Epoch 1 validation: 100%|██████████| 17/17 [00:00<00:00, 32.26it/s]


TRAIN ->	 LOSS: 0.4122,	 AUC: 0.4972,	 AUPRC: 0.1384
VALIDATION ->	 LOSS: 0.3869,	 AUC: 0.5751,	 AUPRC: 0.1453



Epoch 2 validation: 100%|██████████| 17/17 [00:00<00:00, 62.19it/s]


TRAIN ->	 LOSS: 0.3853,	 AUC: 0.4553,	 AUPRC: 0.1162
VALIDATION ->	 LOSS: 0.3791,	 AUC: 0.6325,	 AUPRC: 0.1636



...

TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 1.8888,	 AUC: 0.7041,	 AUPRC: 0.3048



Epoch 499 validation: 100%|██████████| 17/17 [00:00<00:00, 57.80it/s]


TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 1.8919,	 AUC: 0.7031,	 AUPRC: 0.3058



Epoch 500 validation: 100%|██████████| 17/17 [00:00<00:00, 57.45it/s]


TRAIN ->	 LOSS: 0.0000,	 AUC: 1.0000,	 AUPRC: 1.0000
VALIDATION ->	 LOSS: 1.9025,	 AUC: 0.7028,	 AUPRC: 0.3006



# Run with ignite

In [12]:
!pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 5.9 MB/s eta 0:00:00


In [13]:
from ignite.engine import Engine, Events
from ignite.metrics import Loss, ROC_AUC, Precision, Recall, Accuracy
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers.tqdm_logger import ProgressBar
from ignite.metrics import RunningAverage

# Define the trainer
def process_function(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y.unsqueeze(1))
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(process_function)

# Define the evaluator
def eval_function(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        return y_pred, y.unsqueeze(1)

evaluator = Engine(eval_function)

# Metrics
RunningAverage(output_transform=lambda x: x).attach(trainer, "loss")
Loss(criterion).attach(evaluator, 'loss')
ROC_AUC().attach(evaluator, 'roc_auc')
# Precision().attach(evaluator, 'precision')
# Recall().attach(evaluator, 'recall')

# To store predictions and labels for AUPRC calculation
def store_labels_predictions(engine):
    y_pred, y = engine.state.output
    if not hasattr(engine.state, 'y_pred'):
        engine.state.y_pred = []
    if not hasattr(engine.state, 'y_true'):
        engine.state.y_true = []
    engine.state.y_pred.extend(y_pred.cpu())
    engine.state.y_true.extend(y.cpu())
evaluator.add_event_handler(Events.ITERATION_COMPLETED, store_labels_predictions)


# Progress Bar
ProgressBar().attach(trainer)
ProgressBar().attach(evaluator)

# Calculate AUPRC
def calculate_auprc(engine):
    y_pred = torch.cat(engine.state.y_pred).numpy()
    y_true = torch.cat(engine.state.y_true).numpy()
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    auprc = auc(recall, precision)
    return auprc

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    avg_loss = metrics['loss']
    roc_auc = metrics['roc_auc']
    # precision = metrics['precision']
    # recall = metrics['recall']
    auprc = calculate_auprc(evaluator)
    print(f"Training Results - Epoch: {engine.state.epoch}  Avg loss: {avg_loss:.4f}  ROC_AUC: {roc_auc:.4f}  AUPRC: {auprc:.4f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    avg_loss = metrics['loss']
    roc_auc = metrics['roc_auc']
    # precision = metrics['precision']
    # recall = metrics['recall']
    auprc = calculate_auprc(evaluator)
    print(f"Validation Results - Epoch: {engine.state.epoch}  Avg loss: {avg_loss:.4f}  ROC_AUC: {roc_auc:.4f}  AUPRC: {auprc:.4f}")

# Early Stopping
handler = EarlyStopping(patience=10, score_function=lambda engine: engine.state.metrics['roc_auc'], trainer=trainer)
evaluator.add_event_handler(Events.COMPLETED, handler)

# Training
trainer.run(train_loader, max_epochs=20)


/usr/local/lib/python3.10/dist-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
<ipython-input-13-10c2c93f7205>:4: DeprecationWarning: /usr/local/lib/python3.10/dist-packages/ignite/contrib/handlers/tqdm_logger.py has been moved to /ignite/handlers/tqdm_logger.py and will be removed in version 0.6.0.
 Please refer to the documentation for more details.
  from ignite.contrib.handlers.tqdm_logger import ProgressBar
/usr/local/lib/python3.10/dist-packages/ignite/handlers/tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


[1/49]   2%|2          [00:00<?]

[1/49]   2%|2          [00:00<?]

Training Results - Epoch: 1  Avg loss: 0.0000  ROC_AUC: 1.0000  AUPRC: 1.0000


[1/17]   6%|5          [00:00<?]

...

Training Results - Epoch: 6  Avg loss: 0.0000  ROC_AUC: 1.0000  AUPRC: 0.8509


[1/17]   6%|5          [00:00<?]

2024-09-19 08:22:37,008 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Validation Results - Epoch: 6  Avg loss: 1.9277  ROC_AUC: 0.7052  AUPRC: 0.8304


State:
	iteration: 294
	epoch: 6
	epoch_length: 49
	max_epochs: 20
	output: 2.8287308850849513e-07
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

# Move to Drive

In [14]:
!zip -r medical_sam_2_LSTM_embeddings.zip epoch_FLAIR epoch_T1 epoch_T2

  adding: epoch_FLAIR/ (stored 0%)
  adding: epoch_FLAIR/epoch-339.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-410.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-71.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-307.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-389.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-105.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-438.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-122.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-78.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-84.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-145.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-354.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-88.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-65.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-324.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-23.pth (deflated 8%)
  adding: epoch_FLAIR/epoch-275.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-494.pth (deflated 7%)
  adding: epoch_FLAIR/epoch-360.pth (deflated 7%)
  adding: epoch_FLAIR

In [15]:
!du -sh medical_sam_2_LSTM_embeddings.zip

17G	medical_sam_2_LSTM_embeddings.zip


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
!cp medical_sam_2_LSTM_embeddings.zip /content/drive/MyDrive/IAAA2end